In [2]:
from Bio.Seq import Seq
from Bio import SeqIO, Align
import numpy as np
from scipy.sparse import linalg,csr_matrix,csc_matrix,diags
import networkx as nx

In [3]:

def importLongReads(filename):
    ReadSeq={}
    ReadInd=[]
    
    for seq_record in SeqIO.parse(filename, 'fasta'):
        read_id = seq_record.id.split('.')[1]
        ReadSeq[read_id+"_F"] = str(seq_record.seq).upper()
        ReadInd.append(read_id+"_F")
        ReadSeq[read_id+"_R"] = str(Seq(seq_record.seq).reverse_complement()).upper()
        ReadInd.append(read_id+"_R")        
    return(ReadSeq,ReadInd)  

def writeSequence(f, sequence):
    length = len(sequence)
    line=int(length/100)
    for k in range(line):
        f.write(sequence[100*k:100*(k+1)]+'\n')
    if length>line*100:
        f.write(sequence[100*line:]+'\n')

def ParseRecord(record):
    A_ID = record[0].split('.')[1]
    B_ID = record[1].split('.')[1]
    A_Orientation = int(record[2])
    B_Orientation = int(record[3])
    if A_Orientation==0:
        A_name = A_ID+'_F'
    else:
        A_name = A_ID+'_R'
    if B_Orientation==0:
        B_name = B_ID+'_F'
    else:
        B_name = B_ID+'_R'
    return (A_name,B_name)
    
def setupRegressionModelFromMHAP(reads):    
    global edge_dict
    row=0
    indptr=[0]
    indices=[]
    data=[]
    response=[]

    size=len(reads)
    response= []
    for i in range(size-1):
        for j in range(i+1,size):
            A = reads[i]
            B = reads[j]
            key_1 = A+"-"+B
            key_2 = B+"-"+A
            if key_1 in edge_dict:
                for overlap in edge_dict[key_1]:
                    row+=1
                    indices.append(j)
                    data.append(1)
                    indices.append(i)
                    data.append(-1)
                    indptr.append(indptr[-1]+2)
                    response+=[overlap['A_start']-overlap['B_start']]                    
                # print(A,B,response[-1],overlap)                
            if key_2 in edge_dict:
                for overlap in edge_dict[key_2]:
                    row+=1
                    indices.append(i)
                    data.append(1)
                    indices.append(j)
                    data.append(-1)
                    indptr.append(indptr[-1]+2)
                    response+=[overlap['A_start']-overlap['B_start']]
                # print(B,A,response[-1],overlap)           

    design=csr_matrix((data, indices, indptr), shape=(row, size))
    response=np.matrix(response).T

    return(design, response)


In [12]:
def deleteRowsCsr(mat, indices):
    indices = list(indices)
    mask = np.ones(mat.shape[0], dtype=bool)
    mask[indices] = False
    return mat[mask]

#####Huber M-estimate of reads coordinates 
def IRLS(X,Y,reads,perc=90,thr2=50):
    if Y.shape[0]==0:
        return([], [])

    residual_list = []
    t=X.T
    A=t.dot(X)
    y=csr_matrix(Y)
    b=t.dot(y).todense()
    estimate, exitCode = linalg.lgmres(A, b, atol=1e-05) #compute ordinary least squares estiamte
    residual=abs((X.dot(csr_matrix(estimate).T)-y).todense()).T.getA()[0]
    residual_list.append(residual)
    max_residual=max(residual)
    print("Initial max_residual:", round(max_residual,5),"Regression Size:",Y.shape[0])
    # print("Quantile 50 55 60 ... 95:",[np.percentile(residual,5*x+50) for x in range(10)],np.percentile(residual,99))
    thr1 = np.percentile(residual,perc)
    threshold=csr_matrix(np.ones(len(residual))*thr1).toarray()[0] #threshold of Huber's weight function
    old_estimate=estimate
    n=0
    
    while n<100:
        index=np.where(residual>threshold)[0]
        reweight=np.ones(len(residual))
        # print("Quantile 5, 10, ...,50 55 60 ... 95,99:",[np.percentile(residual,5*x) for x in range(1,20)],np.percentile(residual,99))
    
        thr1 = np.percentile(residual,perc) #set thr1 to 90/95 percentile of all residuals
        threshold=csr_matrix(np.ones(len(residual))*thr1).toarray()[0] 
        reweight[index]=threshold[index]/residual[index] # update weights for alignments with residuals greater than thr1
        reweight=diags(reweight)
        t=X.T
        A=t.dot(reweight).dot(X)
        y=csr_matrix(Y)
        b=t.dot(reweight).dot(y).todense()
        estimate, exitCode = linalg.lgmres(A, b, atol=1e-05) #compute weighted least squares estimate
        residual=abs((X.dot(csr_matrix(estimate).T)-y).todense()).T.getA()[0]
        residual_list.append(residual)
        max_residual=max(residual)
        #print("max_residual:", round(max_residual,5))
        diff=max(abs(estimate-old_estimate))
        if diff<1: # convergence condition of estimates
            break
        else:
            old_estimate=estimate
            n+=1
    print("IRLS Stop at the %d rounds with max diff %d, max residual %f"%(n,diff,round(max_residual,5)))
    
    outlier_overlap=[]
    for i in range(len(residual)):        
        n1=X.indices[2*i]
        n2=X.indices[2*i+1]
        if residual[i]>thr2:
            outlier_overlap.append((reads[n1],reads[n2],residual[i]))
    
    index=np.where(residual>thr2)[0]
    X=deleteRowsCsr(X,index)
    Y=np.delete(Y,index,0)
    
    t=X.T
    A=t.dot(X)
    y=csr_matrix(Y)
    b=t.dot(y).todense()
    estimate, exitCode = linalg.lgmres(A, b, atol=1e-05)
    residual_2=abs((X.dot(csr_matrix(estimate).T)-y).todense()).T.getA()[0] ## residuals after step 2
    #max_residual=max(residual)

    #remove alignments with residuals greater than thr2
    G = nx.Graph()
    for i in range(X.shape[0]):
        n1=X.indices[2*i]
        n2=X.indices[2*i+1]
        G.add_edge(n1,n2)

#divide reads into separate connected components
    reads_list=[]
    estimates_list=[]
    for c in nx.connected_components(G):
        sub_index=list(G.subgraph(c).nodes)
        sub_estimates=[estimate[i] for i in sub_index]
        sub_reads=[reads[i] for i in sub_index]
        estimates_list.append(list(map(int,np.round(sub_estimates))))
        reads_list.append(sub_reads)

    return(estimates_list, reads_list, outlier_overlap, residual_list, residual_2)

In [5]:

ReadSeq,ReadInd = importLongReads('./SRR_HIFI_2k.fasta')
HangingOut = 50
G = nx.Graph()
edge_dict = {}
with open("./BLASR_selfmap_2k.Out",'r') as fin:
    for line in fin:
        record = line.strip().split()        
        (A_Orientation,B_Orientation, score)=list(map(int,record[2:5]))
        # if B_Orientation==1:
        #     continue
        perSim = round(float(record[5]),2)
        (B_start, B_end, B_length, A_start, A_end, A_length) = list(map(int,record[6:12]))
        A_name,B_name = ParseRecord(record)
        if A_name==B_name: #or ~(A_name in ReadSeq) or ~(B_name in ReadSeq):
            continue
        overlap={'A_name':A_name,'B_name':B_name,'A_Orientation':A_Orientation,'A_start':A_start,'A_end':A_end,'A_length':A_length,'B_Orientation':B_Orientation,'B_start':B_start,'B_end':B_end,'B_length':B_length,'perSim':perSim}
        key = A_name+"-"+B_name
        if min(A_start,B_start)<HangingOut and min(A_length-A_end,B_length-B_end)<HangingOut:
            if abs((A_end-A_start)-(B_end-B_start))>50:
                continue
            G.add_edge(A_name,B_name)
            if key in edge_dict:
                edge_dict[key].append(overlap)
            else:
                edge_dict[key]=[overlap]
fin.close()

In [13]:
Reads_List = []
Estimate_List = []
Outliers = []
Residual_list = []
Residual_2 =[]
sub_reads_list=[]
for subG in nx.connected_components(G):
    sub_reads = list(G.subgraph(subG).nodes())
    print("SubG.size",len(sub_reads))

    sub_reads_list.append(sub_reads)
    design, response = setupRegressionModelFromMHAP(sub_reads)
    estimates_list, reads_list, outliers, residual_l, residual_2=IRLS(design, response, sub_reads,perc=95)   

    Estimate_List.append(estimates_list)
    Reads_List.append(reads_list)
    Outliers.append(outliers)
    Residual_list.append(residual_l)
    Residual_2.append(residual_2)


SubG.size 3889
Initial max_residual: 53339.47385 Regression Size: 13628


In [11]:
import math

residual_list=Residual_list[0]
residual_final = Residual_2[0]
x = residual_list[0]
y = residual_list[99]
z = residual_final
for i in range(len(x)):
    print(x[i],",",y[i])
    
'''
TGS_residual=open("TGS_residual.csv",'w')
for i in range(len(x)):
    print(x[i],",",y[i],file=TGS_residual)
TGS_residual.close()

Step2_residual = open("TGS_Step2_residual.csv",'w')    
for i in range(len(z)):
    print(z[i],file=Step2_residual)
Step2_residual.close()
'''

959.128507278976 , 2.564974931829056
845.3558471344877 , 1.7641999553488859
314.11725209899305 , 3.8827324680223683
390.78057127080683 , 0.10455874832587142
1935.2477332258204 , 4.148693113223999
1654.675906652912 , 13.084864598644344
2212.1699086826848 , 2.56086906571727
1934.9986994363644 , 4.399041538628808
2213.1680736965063 , 1.5628819000157819
1349.9090785497829 , 2.4604161835031846
2617.804413931888 , 19.6498395304734
924.6513442510623 , 2.4539022635581205
694.6365263739426 , 12.873119303261774
1243.976280500472 , 3.352335458235757
2063.5058743403933 , 1.7224426355787728
1224.1364184052945 , 10.340358792976986
2489.0317537874 , 3.84906455399323
1033.4240043955506 , 1.7453227599617094
1352.7489406449604 , 0.8468895652840729
2172.2785344848817 , 2.476782387941057
73.66331917181378 , 0.7781737196964968
1336.558654553919 , 12.967597066665803
2191.8971036290313 , 4.863855272635192
1987.8822857519117 , 31.55536176706846
3083.7591398702352 , 6.965098995500739
1546.4671619550136 , 2.044

'\nTGS_residual=open("TGS_residual.csv",\'w\')\nfor i in range(len(x)):\n    print(x[i],",",y[i],file=TGS_residual)\nTGS_residual.close()\n\nStep2_residual = open("TGS_Step2_residual.csv",\'w\')    \nfor i in range(len(z)):\n    print(z[i],file=Step2_residual)\nStep2_residual.close()\n'